# Happy Hotel
You've just joined the data team at a large hotelling chain. Your specific team is embedded in the CX (customer experience) division. Each day you receive hundreds of reviews of your 10 hotels from your customers. Each review consists of a free-form text review and a report of "happy" or "not happy". A product manager on your team wants to understand each hotel's performance at a more granular level; they want to build a product to identify topics within reviews. All of your reviews are unlabeled, and it's infeasible to label them by hand.

Design and execute a method to identify topics within the reviews. For each topic, find a robust means of assigning a score to each hotel in that topic. For each of the ten hotels, what recommendations would you make to their general managers?



In [121]:
#dataexploration
import numpy as np
from pandas_profiling import ProfileReport

#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score


#NLP
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
init_notebook_mode(connected=True) #do not miss this line


from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from gensim import corpora, models
from joblib import dump
from gensim.models import CoherenceModel

from gensim import corpora, models
from pprint import pprint

# spacy for lemmatization
import spacy

#plotting
import matplotlib.pyplot as plt
%matplotlib inline
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this



import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sarah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


EDA

In [3]:
happy = pd.read_csv('Happy_Hotel_Data_Set/hotel_happy_reviews.csv')
nothappy = pd.read_csv('Happy_Hotel_Data_Set/hotel_not_happy_reviews.csv')

In [27]:
happy

,User_ID,Description,Is_Response,hotel_ID,Sentiment
0,id10329,Stayed here with husband and sons on the way t...,happy,8,1
1,id10331,We had - rooms. One was very nice and clearly ...,happy,3,1
2,id10333,My wife & I stayed in this glorious city a whi...,happy,3,1
3,id10334,My boyfriend and I stayed at the Fairmont on a...,happy,7,1
4,id10336,"Steps off Times Square, nice rooms, stayed - n...",happy,3,1
...,...,...,...,...,...
26516,id49247,We staed at the Handlery Hotel to Visit San Di...,happy,5,1
26517,id49250,We booked a room with two single beds. Upon ar...,happy,3,1
26518,id49251,Arrived via taxi from La Guardia - driving int...,happy,8,1
26519,id49253,We arrived late at night and walked in to a ch...,happy,8,1


In [28]:
nothappy

,User_ID,Description,Is_Response,hotel_ID,Sentiment
0,id10326,The room was kind of clean but had a VERY stro...,not happy,3,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,not happy,9,0
2,id10328,I booked this hotel through Hotwire at the low...,not happy,3,0
3,id10330,My girlfriends and I stayed here to celebrate ...,not happy,3,0
4,id10332,My husband and I have stayed in this hotel a f...,not happy,7,0
...,...,...,...,...,...
12406,id49249,I am pretty impartial to this hotel. I would d...,not happy,2,0
12407,id49252,I tend to post reviews only when I’ve had an e...,not happy,1,0
12408,id49254,The only positive impression is location and p...,not happy,2,0
12409,id49255,Traveling with friends for shopping and a show...,not happy,5,0


In [18]:
happy.columns

Index(['User_ID', 'Description', 'Is_Response', 'hotel_ID'], dtype='object')

In [23]:
happy['Sentiment']='1'
nothappy['Sentiment']='0'

In [25]:
nothappy

,User_ID,Description,Is_Response,hotel_ID,Sentiment
0,id10326,The room was kind of clean but had a VERY stro...,not happy,3,0
1,id10327,I stayed at the Crown Plaza April -- - April -...,not happy,9,0
2,id10328,I booked this hotel through Hotwire at the low...,not happy,3,0
3,id10330,My girlfriends and I stayed here to celebrate ...,not happy,3,0
4,id10332,My husband and I have stayed in this hotel a f...,not happy,7,0
...,...,...,...,...,...
12406,id49249,I am pretty impartial to this hotel. I would d...,not happy,2,0
12407,id49252,I tend to post reviews only when I’ve had an e...,not happy,1,0
12408,id49254,The only positive impression is location and p...,not happy,2,0
12409,id49255,Traveling with friends for shopping and a show...,not happy,5,0


In [48]:
hotel = pd.concat([happy,nothappy])

In [63]:
hotel.Description[1]

1    We had - rooms. One was very nice and clearly ...
1    I stayed at the Crown Plaza April -- - April -...
Name: Description, dtype: object

In [50]:
hotelprofile=ProfileReport(hotel, title='Happy and Not Happy Hotels', html={'style':{'full_width':True}})

In [ ]:
hotelprofile

Topic analysis of every review

In [65]:
stop_words = stopwords.words('english')

In [112]:
#Create a corpus of reviews
# Convert to list
data = hotel.Description.values.tolist()
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['Stayed here with husband and sons on the way to an Alaska Cruise. We all '
 'loved the hotel, great experience. Ask for a room on the North tower, facing '
 'north west for the best views. We had a high floor, with a stunning view of '
 'the needle, the city, and even the cruise ships! We ordered room service for '
 'dinner so we could enjoy the perfect views. Room service dinners were '
 'delicious, too! You are in a perfect spot to walk everywhere, so enjoy the '
 'city. Almost forgot- Heavenly beds were heavenly, too!']


In [113]:
#Function to find sentences
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [114]:
data_words = list(sent_to_words(data))

print(data_words[:1])

[['stayed', 'here', 'with', 'husband', 'and', 'sons', 'on', 'the', 'way', 'to', 'an', 'alaska', 'cruise', 'we', 'all', 'loved', 'the', 'hotel', 'great', 'experience', 'ask', 'for', 'room', 'on', 'the', 'north', 'tower', 'facing', 'north', 'west', 'for', 'the', 'best', 'views', 'we', 'had', 'high', 'floor', 'with', 'stunning', 'view', 'of', 'the', 'needle', 'the', 'city', 'and', 'even', 'the', 'cruise', 'ships', 'we', 'ordered', 'room', 'service', 'for', 'dinner', 'so', 'we', 'could', 'enjoy', 'the', 'perfect', 'views', 'room', 'service', 'dinners', 'were', 'delicious', 'too', 'you', 'are', 'in', 'perfect', 'spot', 'to', 'walk', 'everywhere', 'so', 'enjoy', 'the', 'city', 'almost', 'forgot', 'heavenly', 'beds', 'were', 'heavenly', 'too']]


In [115]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['stayed', 'here', 'with', 'husband', 'and', 'sons', 'on', 'the', 'way', 'to', 'an', 'alaska_cruise', 'we', 'all', 'loved', 'the', 'hotel', 'great', 'experience', 'ask', 'for', 'room', 'on', 'the', 'north', 'tower', 'facing', 'north', 'west', 'for', 'the', 'best', 'views', 'we', 'had', 'high', 'floor', 'with', 'stunning', 'view', 'of', 'the', 'needle', 'the', 'city', 'and', 'even', 'the', 'cruise_ships', 'we', 'ordered', 'room', 'service', 'for', 'dinner', 'so', 'we', 'could', 'enjoy', 'the', 'perfect', 'views', 'room', 'service', 'dinners', 'were', 'delicious', 'too', 'you', 'are', 'in', 'perfect', 'spot', 'to', 'walk', 'everywhere', 'so', 'enjoy', 'the', 'city', 'almost', 'forgot', 'heavenly', 'beds', 'were', 'heavenly', 'too']


In [117]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [119]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


In [122]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [123]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['stay', 'husband', 'son', 'alaska_cruise', 'love', 'hotel', 'great', 'experience', 'ask', 'face', 'good', 'view', 'high', 'floor', 'stunning', 'view', 'city', 'even', 'order', 'room', 'service', 'dinner', 'could', 'enjoy', 'perfect', 'view', 'room', 'service', 'dinner', 'delicious', 'perfect', 'spot', 'walk', 'everywhere', 'enjoy', 'city', 'almost', 'forget', 'heavenly', 'bed', 'heavenly']]


In [125]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

In [126]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [127]:

# View
print(corpus[:1])
id2word[0]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1)]]


'alaska_cruise'

In [143]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

KeyboardInterrupt: 

## TF-IDF LDA model

In [ ]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
# Build TF-IDF LDA model
#back to the model


lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## Data visualization 

In [141]:
hotel.columns

Index(['User_ID', 'Description', 'Is_Response', 'hotel_ID', 'Sentiment'], dtype='object')

In [142]:
hotel

,User_ID,Description,Is_Response,hotel_ID,Sentiment
0,id10329,Stayed here with husband and sons on the way t...,happy,8,1
1,id10331,We had - rooms. One was very nice and clearly ...,happy,3,1
2,id10333,My wife & I stayed in this glorious city a whi...,happy,3,1
3,id10334,My boyfriend and I stayed at the Fairmont on a...,happy,7,1
4,id10336,"Steps off Times Square, nice rooms, stayed - n...",happy,3,1
...,...,...,...,...,...
12406,id49249,I am pretty impartial to this hotel. I would d...,not happy,2,0
12407,id49252,I tend to post reviews only when I’ve had an e...,not happy,1,0
12408,id49254,The only positive impression is location and p...,not happy,2,0
12409,id49255,Traveling with friends for shopping and a show...,not happy,5,0
